In [1]:
import numpy as np
import pandas as pd

## Hierarchical indexing

Valuable when working with multidimensional data

In [2]:
toy_data = pd.Series(np.random.randn(12),index=[['a','a','a','a','a','b','b','b','b','b','c','c'],
                                                [1,2,3,4,5,1,2,3,4,5,3,4]])
toy_data

a  1   -1.169925
   2   -0.276510
   3    0.255112
   4   -1.842960
   5    1.331130
b  1    0.222369
   2    0.066668
   3    0.573017
   4    1.857103
   5    0.520230
c  3   -1.145171
   4    0.159143
dtype: float64

In [3]:
toy_data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('a', 4),
            ('a', 5),
            ('b', 1),
            ('b', 2),
            ('b', 3),
            ('b', 4),
            ('b', 5),
            ('c', 3),
            ('c', 4)],
           )

In [4]:
toy_data.loc['a']

1   -1.169925
2   -0.276510
3    0.255112
4   -1.842960
5    1.331130
dtype: float64

We can select the inner level

In [5]:
toy_data.loc[:,3]

a    0.255112
b    0.573017
c   -1.145171
dtype: float64

In [6]:
toy_data.unstack()

,1,2,3,4,5
a,-1.169925,-0.276510,0.255112,-1.842960,1.33113
b,0.222369,0.066668,0.573017,1.857103,0.52023
c,NaN,NaN,-1.145171,0.159143,NaN


In [7]:
toy_data.unstack().stack()

a  1   -1.169925
   2   -0.276510
   3    0.255112
   4   -1.842960
   5    1.331130
b  1    0.222369
   2    0.066668
   3    0.573017
   4    1.857103
   5    0.520230
c  3   -1.145171
   4    0.159143
dtype: float64

## Reshaping and pivoting

In [9]:
df = pd.read_csv("pivot_toy.txt",sep="\t" )
df

,Gene,Status,Expression,Colour
0,gene_01,susceptible,100,Red
1,gene_02,susceptible,120,Red
2,gene_03,susceptible,160,Red
3,gene_04,susceptible,222,Red
4,gene_05,susceptible,190,Red
5,gene_01,resistant,185,Blue
6,gene_02,resistant,200,Blue
7,gene_03,resistant,230,Blue
8,gene_04,resistant,180,Blue
9,gene_05,resistant,250,Blue


In [10]:
df.stack()

0  Gene              gene_01
   Status        susceptible
   Expression            100
   Colour                Red
1  Gene              gene_02
   Status        susceptible
   Expression            120
   Colour                Red
2  Gene              gene_03
   Status        susceptible
   Expression            160
   Colour                Red
3  Gene              gene_04
   Status        susceptible
   Expression            222
   Colour                Red
4  Gene              gene_05
   Status        susceptible
   Expression            190
   Colour                Red
5  Gene              gene_01
   Status          resistant
   Expression            185
   Colour               Blue
6  Gene              gene_02
   Status          resistant
   Expression            200
   Colour               Blue
7  Gene              gene_03
   Status          resistant
   Expression            230
   Colour               Blue
8  Gene              gene_04
   Status          resistant
   Expression 

In [11]:
df.stack().unstack()

,Gene,Status,Expression,Colour
0,gene_01,susceptible,100,Red
1,gene_02,susceptible,120,Red
2,gene_03,susceptible,160,Red
3,gene_04,susceptible,222,Red
4,gene_05,susceptible,190,Red
5,gene_01,resistant,185,Blue
6,gene_02,resistant,200,Blue
7,gene_03,resistant,230,Blue
8,gene_04,resistant,180,Blue
9,gene_05,resistant,250,Blue


In [12]:
df.pivot(index='Gene',columns='Status',values='Expression')

Status,resistant,susceptible
Gene,,
gene_01,185,100
gene_02,200,120
gene_03,230,160
gene_04,180,222
gene_05,250,190


What if we want to use more than 1 column. We have an enhanced function named `pivot_table`

In [13]:
df.pivot_table(index='Gene',columns=['Status','Colour'],values='Expression')

Status,resistant,susceptible
Colour,Blue,Red
Gene,,
gene_01,185,100
gene_02,200,120
gene_03,230,160
gene_04,180,222
gene_05,250,190


In [14]:
df.pivot_table(index=['Status','Colour'],columns=['Gene'],values='Expression')

,Gene,gene_01,gene_02,gene_03,gene_04,gene_05
Status,Colour,,,,,
resistant,Blue,185,200,230,180,250
susceptible,Red,100,120,160,222,190


In [15]:
df.pivot_table(index=['Colour','Status'],columns=['Gene'],values='Expression')

,Gene,gene_01,gene_02,gene_03,gene_04,gene_05
Colour,Status,,,,,
Blue,resistant,185,200,230,180,250
Red,susceptible,100,120,160,222,190


In [16]:
df.pivot_table(index=['Colour','Status'],columns=['Gene'],values='Expression').swaplevel(0,1)

,Gene,gene_01,gene_02,gene_03,gene_04,gene_05
Status,Colour,,,,,
resistant,Blue,185,200,230,180,250
susceptible,Red,100,120,160,222,190


A reverse process to `pivot` can be achieved using `melt`

In [18]:
df_wide = df.pivot_table(index=['Status','Colour'],columns=['Gene'],values='Expression')
df_wide.reset_index(level=['Status','Colour'],inplace=True)
#df_wide.columns.name = None
df_wide

Gene,Status,Colour,gene_01,gene_02,gene_03,gene_04,gene_05
0,resistant,Blue,185,200,230,180,250
1,susceptible,Red,100,120,160,222,190


In [19]:
pd.melt(df_wide, id_vars=['Status','Colour'],
        value_vars=['gene_01','gene_02','gene_03','gene_04','gene_05'],
        var_name='Gene',value_name='Gene_expression')

,Status,Colour,Gene,Gene_expression
0,resistant,Blue,gene_01,185
1,susceptible,Red,gene_01,100
2,resistant,Blue,gene_02,200
3,susceptible,Red,gene_02,120
4,resistant,Blue,gene_03,230
5,susceptible,Red,gene_03,160
6,resistant,Blue,gene_04,180
7,susceptible,Red,gene_04,222
8,resistant,Blue,gene_05,250
9,susceptible,Red,gene_05,190


In [20]:
pd.melt(df_wide, id_vars=['Status','Colour'],
        var_name='Gene',value_name='Gene_expression')

,Status,Colour,Gene,Gene_expression
0,resistant,Blue,gene_01,185
1,susceptible,Red,gene_01,100
2,resistant,Blue,gene_02,200
3,susceptible,Red,gene_02,120
4,resistant,Blue,gene_03,230
5,susceptible,Red,gene_03,160
6,resistant,Blue,gene_04,180
7,susceptible,Red,gene_04,222
8,resistant,Blue,gene_05,250
9,susceptible,Red,gene_05,190


In [21]:
pd.melt(df_wide,id_vars=['Status','Colour'],value_vars='gene_04',
       var_name='Gene',value_name='Gene_expression')

,Status,Colour,Gene,Gene_expression
0,resistant,Blue,gene_04,180
1,susceptible,Red,gene_04,222


##  Sorting

In [2]:
pheno = pd.read_csv("ac_pheno.txt", sep="\t")
pheno.head(10)

,PIT,Length,Weight,Tank,Sex,Site
0,919540,465.0,1514.0,1.0,U,1.0
1,918025,455.0,1250.0,1.0,U,1.0
2,917803,405.0,937.0,1.0,U,1.0
3,918763,505.0,2667.0,4.0,M,2.0
4,917365,500.0,2204.0,4.0,U,2.0
5,916380,520.0,2336.0,4.0,U,2.0
6,9186524,535.0,3065.0,4.0,U,2.0
7,915778,490.0,1774.0,4.0,U,2.0
8,916993,435.0,1426.0,3.0,U,1.0
9,916238,475.0,1545.0,3.0,U,1.0


In [26]:
pheno.describe()

,PIT,Length,Weight,Tank,Site
count,2.862000e+03,2838.000000,2845.000000,2852.000000,2852.000000
mean,9.527062e+05,463.836152,1648.482250,2.934432,1.461781
std,5.343191e+05,39.908756,529.221099,1.155825,0.498625
min,9.155810e+05,305.000000,306.000000,1.000000,1.000000
25%,9.167982e+05,440.000000,1286.000000,2.000000,1.000000
50%,9.180005e+05,465.000000,1600.000000,3.000000,1.000000
75%,9.193178e+05,490.000000,1950.000000,4.000000,2.000000
max,9.202514e+06,590.000000,3942.000000,4.000000,2.000000


In [27]:
pheno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2862 entries, 0 to 2861
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PIT     2862 non-null   int64  
 1   Length  2838 non-null   float64
 2   Weight  2845 non-null   float64
 3   Tank    2852 non-null   float64
 4   Sex     2852 non-null   object 
 5   Site    2852 non-null   float64
dtypes: float64(4), int64(1), object(1)
memory usage: 134.3+ KB


In [23]:
pheno[['PIT','Tank','Sex','Site']].astype('str').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2862 entries, 0 to 2861
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PIT     2862 non-null   object
 1   Tank    2862 non-null   object
 2   Sex     2862 non-null   object
 3   Site    2862 non-null   object
dtypes: object(4)
memory usage: 89.6+ KB


In [29]:
pheno.head(20)

,PIT,Length,Weight,Tank,Sex,Site
0,919540,465.0,1514.0,1.0,U,1.0
1,918025,455.0,1250.0,1.0,U,1.0
2,917803,405.0,937.0,1.0,U,1.0
3,918763,505.0,2667.0,4.0,M,2.0
4,917365,500.0,2204.0,4.0,U,2.0
5,916380,520.0,2336.0,4.0,U,2.0
6,9186524,535.0,3065.0,4.0,U,2.0
7,915778,490.0,1774.0,4.0,U,2.0
8,916993,435.0,1426.0,3.0,U,1.0
9,916238,475.0,1545.0,3.0,U,1.0


In [30]:
pheno.sort_values('Weight', ascending=True).head(20)

,PIT,Length,Weight,Tank,Sex,Site
559,916928,305.0,306.0,1.0,U,1.0
885,917149,330.0,375.0,2.0,U,1.0
991,917019,315.0,388.0,4.0,U,2.0
1382,917995,325.0,406.0,3.0,U,1.0
1462,917050,335.0,494.0,3.0,U,1.0
1902,917553,330.0,496.0,4.0,U,2.0
924,916880,345.0,502.0,2.0,U,1.0
1497,918281,330.0,512.0,3.0,U,1.0
1124,918088,330.0,522.0,3.0,U,1.0
203,915824,350.0,531.0,1.0,U,1.0


In [31]:
pheno.sort_values('Weight',ascending=False).head(20)

,PIT,Length,Weight,Tank,Sex,Site
1804,920039,550.0,3942.0,4.0,M,2.0
2159,920275,565.0,3773.0,4.0,M,2.0
1793,919642,545.0,3719.0,4.0,U,2.0
981,919142,550.0,3712.0,4.0,U,2.0
2011,920263,585.0,3675.0,4.0,M,2.0
2586,918565,560.0,3650.0,4.0,M,2.0
1986,919961,555.0,3568.0,4.0,M,2.0
1831,915904,590.0,3561.0,4.0,M,2.0
2590,919272,520.0,3525.0,4.0,U,2.0
1694,917763,580.0,3460.0,4.0,U,2.0


In [24]:
pheno.sort_values(['Weight','Length'],ascending=[True,True]).head()

,PIT,Length,Weight,Tank,Sex,Site
559,916928,305.0,306.0,1.0,U,1.0
885,917149,330.0,375.0,2.0,U,1.0
991,917019,315.0,388.0,4.0,U,2.0
1382,917995,325.0,406.0,3.0,U,1.0
1462,917050,335.0,494.0,3.0,U,1.0


In [33]:
pheno.head(20)

,PIT,Length,Weight,Tank,Sex,Site
0,919540,465.0,1514.0,1.0,U,1.0
1,918025,455.0,1250.0,1.0,U,1.0
2,917803,405.0,937.0,1.0,U,1.0
3,918763,505.0,2667.0,4.0,M,2.0
4,917365,500.0,2204.0,4.0,U,2.0
5,916380,520.0,2336.0,4.0,U,2.0
6,9186524,535.0,3065.0,4.0,U,2.0
7,915778,490.0,1774.0,4.0,U,2.0
8,916993,435.0,1426.0,3.0,U,1.0
9,916238,475.0,1545.0,3.0,U,1.0


In [34]:
pheno.sort_values(['Weight','Length'],ascending=[True,True], inplace=True)

In [35]:
pheno.head(20)

,PIT,Length,Weight,Tank,Sex,Site
559,916928,305.0,306.0,1.0,U,1.0
885,917149,330.0,375.0,2.0,U,1.0
991,917019,315.0,388.0,4.0,U,2.0
1382,917995,325.0,406.0,3.0,U,1.0
1462,917050,335.0,494.0,3.0,U,1.0
1902,917553,330.0,496.0,4.0,U,2.0
924,916880,345.0,502.0,2.0,U,1.0
1497,918281,330.0,512.0,3.0,U,1.0
1124,918088,330.0,522.0,3.0,U,1.0
203,915824,350.0,531.0,1.0,U,1.0


## Grouping - Summarising data

In [3]:
grouped = pheno.groupby('Site')

In [4]:
grouped

That was not so informative.

In [7]:
grouped.groups.keys()

dict_keys([1.0, 2.0])

We can access each group separately if needed.

In [8]:
grouped.get_group(1)

,PIT,Length,Weight,Tank,Sex,Site
0,919540,465.0,1514.0,1.0,U,1.0
1,918025,455.0,1250.0,1.0,U,1.0
2,917803,405.0,937.0,1.0,U,1.0
8,916993,435.0,1426.0,3.0,U,1.0
9,916238,475.0,1545.0,3.0,U,1.0
...,...,...,...,...,...,...
2857,920227,475.0,1565.0,2.0,U,1.0
2858,915897,440.0,1300.0,2.0,U,1.0
2859,917341,450.0,1510.0,2.0,U,1.0
2860,915861,450.0,1464.0,2.0,U,1.0


In [25]:
pheno.groupby('Site').size()

Site
1.0    1535
2.0    1317
dtype: int64

In [26]:
pheno.groupby(['Site','Tank']).size()

Site  Tank
1.0   1.0      500
      2.0      504
      3.0      531
2.0   4.0     1317
dtype: int64

We can even use customed functions

In [27]:
pheno.groupby(pheno.apply(lambda x: 'large' 
                          if x['Weight'] > 1400 else 'small', axis=1)).size()

large    1867
small     995
dtype: int64

Even more elaborate grouping

In [28]:
pheno.groupby(['Site','Tank', pheno.apply(lambda x: 'large' 
                                          if x['Weight'] > 1400 else 'small', axis=1)]).size()

Site  Tank       
1.0   1.0   large     249
            small     251
      2.0   large     297
            small     207
      3.0   large     276
            small     255
2.0   4.0   large    1045
            small     272
dtype: int64

Often the above patterns can be combined with filtering

In [29]:
pheno.groupby(['Site','Tank','Sex']).size()

Site  Tank  Sex
1.0   1.0   M        95
            U       405
      2.0   M       113
            U       391
      3.0   M       131
            U       400
2.0   4.0   M       132
            U      1185
dtype: int64

In [30]:
pheno_filtered = pheno.groupby(['Site','Tank','Sex']).filter(lambda x: len(x) > 200)
pheno_filtered.head()

,PIT,Length,Weight,Tank,Sex,Site
0,919540,465.0,1514.0,1.0,U,1.0
1,918025,455.0,1250.0,1.0,U,1.0
2,917803,405.0,937.0,1.0,U,1.0
4,917365,500.0,2204.0,4.0,U,2.0
5,916380,520.0,2336.0,4.0,U,2.0


In [31]:
pheno_filtered.groupby(['Site','Tank','Sex']).size()

Site  Tank  Sex
1.0   1.0   U       405
      2.0   U       391
      3.0   U       400
2.0   4.0   U      1185
dtype: int64

It is easy to get summary statistics 

In [32]:
pheno.groupby('Site').mean()

,PIT,Length,Weight,Tank
Site,,,,
1.0,961106.250814,450.372781,1445.198953,2.020195
2.0,943181.753986,479.384966,1884.334093,4.000000


In [33]:
pheno.groupby('Tank')[["Length","Weight"]].aggregate(['max','min',np.mean,np.median,np.std])

Length                                       Weight                      \
        max    min        mean median        std     max    min         mean   
Tank                                                                           
1.0   535.0  305.0  447.748479  450.0  35.488771  2943.0  306.0  1412.042596   
2.0   560.0  330.0  453.233533  455.0  34.785390  2654.0  375.0  1476.640873   
3.0   535.0  325.0  450.108159  455.0  36.074301  2894.0  406.0  1446.139360   
4.0   590.0  315.0  479.384966  485.0  39.059049  3942.0  388.0  1884.334093   

                          
      median         std  
Tank                      
1.0   1401.0  380.831381  
2.0   1470.5  381.479994  
3.0   1426.0  398.821031  
4.0   1852.0  571.773224

In [34]:
pheno.groupby(['Site','Tank'])[["Length","Weight"]].aggregate(['max','min',np.mean,np.median,np.std])

Length                                       Weight         \
             max    min        mean median        std     max    min   
Site Tank                                                              
1.0  1.0   535.0  305.0  447.748479  450.0  35.488771  2943.0  306.0   
     2.0   560.0  330.0  453.233533  455.0  34.785390  2654.0  375.0   
     3.0   535.0  325.0  450.108159  455.0  36.074301  2894.0  406.0   
2.0  4.0   590.0  315.0  479.384966  485.0  39.059049  3942.0  388.0   

                                            
                  mean  median         std  
Site Tank                                   
1.0  1.0   1412.042596  1401.0  380.831381  
     2.0   1476.640873  1470.5  381.479994  
     3.0   1446.139360  1426.0  398.821031  
2.0  4.0   1884.334093  1852.0  571.773224

## Merging datasets

### Joining dataframes with `concat`

In [35]:
s1 = pd.Series(np.arange(10))
s2 = pd.Series(np.arange(11,20))
s3 = pd.Series(np.arange(21,31))

In [56]:
s1

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int64

In [57]:
s2

0    11
1    12
2    13
3    14
4    15
5    16
6    17
7    18
8    19
dtype: int64

In [58]:
s3

0    21
1    22
2    23
3    24
4    25
5    26
6    27
7    28
8    29
9    30
dtype: int64

In [36]:
pd.concat([s1,s2,s3])

0     0
1     1
2     2
3     3
4     4
5     5
6     6
7     7
8     8
9     9
0    11
1    12
2    13
3    14
4    15
5    16
6    17
7    18
8    19
0    21
1    22
2    23
3    24
4    25
5    26
6    27
7    28
8    29
9    30
dtype: int64

Create a new index

In [37]:
pd.concat([s1,s2,s3],ignore_index=True)

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    11
11    12
12    13
13    14
14    15
15    16
16    17
17    18
18    19
19    21
20    22
21    23
22    24
23    25
24    26
25    27
26    28
27    29
28    30
dtype: int64

Join columnwise

In [38]:
pd.concat([s1,s2,s3],axis=1)

,0,1,2
0,0,11.0,21
1,1,12.0,22
2,2,13.0,23
3,3,14.0,24
4,4,15.0,25
5,5,16.0,26
6,6,17.0,27
7,7,18.0,28
8,8,19.0,29
9,9,NaN,30


Create a hierachical index

In [39]:
pd.concat([s1,s2,s3],keys=['a','b','c'])

a  0     0
   1     1
   2     2
   3     3
   4     4
   5     5
   6     6
   7     7
   8     8
   9     9
b  0    11
   1    12
   2    13
   3    14
   4    15
   5    16
   6    17
   7    18
   8    19
c  0    21
   1    22
   2    23
   3    24
   4    25
   5    26
   6    27
   7    28
   8    29
   9    30
dtype: int64

In [40]:
pd.concat([s1,s2,s3],keys=['a','b','c']).unstack()

,0,1,2,3,4,5,6,7,8,9
a,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
b,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,NaN
c,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0


We can use `concat` to join dataframes

In [41]:
pedigree = pd.read_csv("ac_ped.txt", sep="\t")
pedigree.head()

,Id,Sire,Dam,Year_Class,Selected_gen
0,478665,0,0,2013,7
1,478620,0,0,2013,7
2,478601,02F49B,01FD38,2013,7
3,478656,02F49B,01FD38,2013,7
4,478671,02F49B,01FD38,2013,7


In [42]:
pedigree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8362 entries, 0 to 8361
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            8362 non-null   int64 
 1   Sire          8362 non-null   object
 2   Dam           8362 non-null   object
 3   Year_Class    8362 non-null   int64 
 4   Selected_gen  8362 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 326.8+ KB


In [43]:
pd.concat([pedigree, pheno],axis=1)

,Id,Sire,Dam,Year_Class,Selected_gen,PIT,Length,Weight,Tank,Sex,Site
0,478665,0,0,2013,7,919540.0,465.0,1514.0,1.0,U,1.0
1,478620,0,0,2013,7,918025.0,455.0,1250.0,1.0,U,1.0
2,478601,02F49B,01FD38,2013,7,917803.0,405.0,937.0,1.0,U,1.0
3,478656,02F49B,01FD38,2013,7,918763.0,505.0,2667.0,4.0,M,2.0
4,478671,02F49B,01FD38,2013,7,917365.0,500.0,2204.0,4.0,U,2.0
...,...,...,...,...,...,...,...,...,...,...,...
8357,919218,597579,900730,2017,8,NaN,NaN,NaN,NaN,NaN,NaN
8358,919013,597579,900730,2017,8,NaN,NaN,NaN,NaN,NaN,NaN
8359,919448,597579,900730,2017,8,NaN,NaN,NaN,NaN,NaN,NaN
8360,919246,597579,900730,2017,8,NaN,NaN,NaN,NaN,NaN,NaN


The previous though is not really what we wanted to. We need to assign as index the common column in both dataframes

In [44]:
pheno_index_PIT = pheno.set_index('PIT')
pheno_index_PIT.rename_axis(index={'PIT':""},inplace=True)
pheno_index_PIT = pheno_index_PIT.loc[~pheno_index_PIT.index.duplicated(),:]
pheno_index_PIT.sort_index(inplace=True)
pheno_index_PIT

,Length,Weight,Tank,Sex,Site
,,,,,
915581,460.0,1470.0,2.0,U,1.0
915582,450.0,1752.0,4.0,U,2.0
915583,440.0,1436.0,2.0,U,1.0
915584,465.0,1585.0,4.0,U,2.0
915587,465.0,1594.0,4.0,U,2.0
...,...,...,...,...,...
9195532,430.0,1847.0,2.0,U,1.0
9196545,485.0,1713.0,1.0,U,1.0
9197101,420.0,1069.0,2.0,U,1.0


In [45]:
pedigree_index_Id = pedigree.set_index('Id')
pedigree_index_Id.rename_axis(index={'Id':''},inplace=True)
pedigree_index_Id = pedigree_index_Id.loc[~pedigree_index_Id.index.duplicated(),:]
pedigree_index_Id.sort_index(inplace=True)
pedigree_index_Id

,Sire,Dam,Year_Class,Selected_gen
,,,,
478569,02F49B,01FD38,2013,7
478572,02F49B,01FD38,2013,7
478573,02F49B,01FD38,2013,7
478574,02F49B,01FD38,2013,7
478575,02F49B,01FD38,2013,7
...,...,...,...,...
920576,478788,480166,2017,8
920577,480075,597462,2017,8
920578,480450,480112,2017,8


In [46]:
pd.concat([pedigree_index_Id,pheno_index_PIT],axis=1,join='inner')

,Sire,Dam,Year_Class,Selected_gen,Length,Weight,Tank,Sex,Site
,,,,,,,,,
915581,597426,480449,2017,8,460.0,1470.0,2.0,U,1.0
915582,597179,480301,2017,8,450.0,1752.0,4.0,U,2.0
915583,900663,596937,2017,8,440.0,1436.0,2.0,U,1.0
915584,479924,597166,2017,8,465.0,1585.0,4.0,U,2.0
915587,479557,596943,2017,8,465.0,1594.0,4.0,U,2.0
...,...,...,...,...,...,...,...,...,...
920570,900741,479406,2017,8,505.0,2079.0,2.0,U,1.0
920571,480439,597565,2017,8,440.0,1487.0,2.0,M,1.0
920574,480535,480155,2017,8,470.0,1520.0,4.0,U,2.0


We can also create a hierachical dataframe

In [47]:
pd.concat([pedigree_index_Id, pheno_index_PIT],axis=1, join='inner', keys=['A','B'])

A                                      B                      
          Sire     Dam Year_Class Selected_gen Length  Weight Tank Sex Site
                                                                           
915581  597426  480449       2017            8  460.0  1470.0  2.0   U  1.0
915582  597179  480301       2017            8  450.0  1752.0  4.0   U  2.0
915583  900663  596937       2017            8  440.0  1436.0  2.0   U  1.0
915584  479924  597166       2017            8  465.0  1585.0  4.0   U  2.0
915587  479557  596943       2017            8  465.0  1594.0  4.0   U  2.0
...        ...     ...        ...          ...    ...     ...  ...  ..  ...
920570  900741  479406       2017            8  505.0  2079.0  2.0   U  1.0
920571  480439  597565       2017            8  440.0  1487.0  2.0   M  1.0
920574  480535  480155       2017            8  470.0  1520.0  4.0   U  2.0
920577  480075  597462       2017            8  460.0  1313.0  3.0   M  1.0
920578  480450  480112       2017            8  495.0  2253.0  4.0   U  2.0

[2780 rows x 9 columns]

### Joining files with `merge`

Another option is to use `merge` when we need to join dataframes.First we will try a union of the two files

In [48]:
pedigree['Id'] = pedigree['Id'].astype('str')
pheno['PIT'] = pheno['PIT'].astype('str')

In [49]:
combine = pd.merge(pedigree, pheno, left_on = "Id", right_on = "PIT", how = "outer")
combine

,Id,Sire,Dam,Year_Class,Selected_gen,PIT,Length,Weight,Tank,Sex,Site
0,478665,0,0,2013.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN
1,478620,0,0,2013.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN
2,478601,02F49B,01FD38,2013.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN
3,478656,02F49B,01FD38,2013.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN
4,478671,02F49B,01FD38,2013.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8439,NaN,NaN,NaN,NaN,NaN,9186604,520.0,2026.0,4.0,U,2.0
8440,NaN,NaN,NaN,NaN,NaN,9198621,490.0,2242.0,4.0,M,2.0
8441,NaN,NaN,NaN,NaN,NaN,920254,555.0,2983.0,4.0,U,2.0
8442,NaN,NaN,NaN,NaN,NaN,9195532,430.0,1847.0,2.0,U,1.0


Only the key combinations found in the left dataframe

In [50]:
combine_left = pd.merge(pedigree, pheno, left_on = "Id", right_on = "PIT", how = "left")
combine_left

,Id,Sire,Dam,Year_Class,Selected_gen,PIT,Length,Weight,Tank,Sex,Site
0,478665,0,0,2013,7,NaN,NaN,NaN,NaN,NaN,NaN
1,478620,0,0,2013,7,NaN,NaN,NaN,NaN,NaN,NaN
2,478601,02F49B,01FD38,2013,7,NaN,NaN,NaN,NaN,NaN,NaN
3,478656,02F49B,01FD38,2013,7,NaN,NaN,NaN,NaN,NaN,NaN
4,478671,02F49B,01FD38,2013,7,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8424,919218,597579,900730,2017,8,NaN,NaN,NaN,NaN,NaN,NaN
8425,919013,597579,900730,2017,8,NaN,NaN,NaN,NaN,NaN,NaN
8426,919448,597579,900730,2017,8,NaN,NaN,NaN,NaN,NaN,NaN
8427,919246,597579,900730,2017,8,NaN,NaN,NaN,NaN,NaN,NaN


Only the key combinations found in the right dataframe

In [51]:
combine_right = pd.merge(pedigree, pheno, left_on = "Id", right_on = "PIT", how = "right")
combine_right

,Id,Sire,Dam,Year_Class,Selected_gen,PIT,Length,Weight,Tank,Sex,Site
0,919540,597579,900730,2017.0,8.0,919540,465.0,1514.0,1.0,U,1.0
1,918025,480075,597462,2017.0,8.0,918025,455.0,1250.0,1.0,U,1.0
2,917803,479785,479061,2017.0,8.0,917803,405.0,937.0,1.0,U,1.0
3,918763,596713,480633,2017.0,8.0,918763,505.0,2667.0,4.0,M,2.0
4,917365,478795,479821,2017.0,8.0,917365,500.0,2204.0,4.0,U,2.0
...,...,...,...,...,...,...,...,...,...,...,...
2857,920227,479024,597516,2017.0,8.0,920227,475.0,1565.0,2.0,U,1.0
2858,915897,597426,480449,2017.0,8.0,915897,440.0,1300.0,2.0,U,1.0
2859,917341,478795,479821,2017.0,8.0,917341,450.0,1510.0,2.0,U,1.0
2860,915861,597426,480449,2017.0,8.0,915861,450.0,1464.0,2.0,U,1.0


The resulting file contains only the common keys

In [52]:
combine_inner = pd.merge(pedigree, pheno, left_on = "Id", right_on = "PIT", how = "inner")
combine_inner

,Id,Sire,Dam,Year_Class,Selected_gen,PIT,Length,Weight,Tank,Sex,Site
0,916577,597579,479801,2017,8,916577,455.0,1556.0,3.0,U,1.0
1,915812,597579,479801,2017,8,915812,430.0,1339.0,3.0,M,1.0
2,915812,597579,479801,2017,8,915812,430.0,1339.0,3.0,M,1.0
3,916294,597579,479801,2017,8,916294,505.0,2038.0,2.0,M,1.0
4,916246,597579,479801,2017,8,916246,415.0,1213.0,1.0,U,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2842,918721,597579,900730,2017,8,918721,500.0,2168.0,4.0,U,2.0
2843,919049,597579,900730,2017,8,919049,530.0,2862.0,4.0,U,2.0
2844,918919,597579,900730,2017,8,918919,505.0,1888.0,4.0,U,2.0
2845,918637,597579,900730,2017,8,918637,430.0,1208.0,4.0,U,2.0


Below we see an example of how we can perform a so called semi_join. In other words filter a dataframe based on ids found in another dataframe by instead of the previous merges we keep only the columns of the dataframe of interest

In [87]:
pedigree['Id'].isin(pheno['PIT'])

0       False
1       False
2       False
3       False
4       False
        ...  
8357    False
8358    False
8359    False
8360    False
8361    False
Name: Id, Length: 8362, dtype: bool

In [53]:
pedigree.loc[pedigree['Id'].isin(pheno['PIT'])]

,Id,Sire,Dam,Year_Class,Selected_gen
3656,916577,597579,479801,2017,8
3657,915812,597579,479801,2017,8
3658,916294,597579,479801,2017,8
3659,916246,597579,479801,2017,8
3660,916009,597579,479801,2017,8
...,...,...,...,...,...
8326,918721,597579,900730,2017,8
8327,919049,597579,900730,2017,8
8328,918919,597579,900730,2017,8
8329,918637,597579,900730,2017,8


For all the previous `merge` examples we can also use a different syntax that is also useful for performing more advanced `merge` joins 

In [54]:
pedigree.merge(pheno,left_on="Id",right_on="PIT",how="inner" )

,Id,Sire,Dam,Year_Class,Selected_gen,PIT,Length,Weight,Tank,Sex,Site
0,916577,597579,479801,2017,8,916577,455.0,1556.0,3.0,U,1.0
1,915812,597579,479801,2017,8,915812,430.0,1339.0,3.0,M,1.0
2,915812,597579,479801,2017,8,915812,430.0,1339.0,3.0,M,1.0
3,916294,597579,479801,2017,8,916294,505.0,2038.0,2.0,M,1.0
4,916246,597579,479801,2017,8,916246,415.0,1213.0,1.0,U,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2842,918721,597579,900730,2017,8,918721,500.0,2168.0,4.0,U,2.0
2843,919049,597579,900730,2017,8,919049,530.0,2862.0,4.0,U,2.0
2844,918919,597579,900730,2017,8,918919,505.0,1888.0,4.0,U,2.0
2845,918637,597579,900730,2017,8,918637,430.0,1208.0,4.0,U,2.0


The above syntax can be used to join more than 2 dataframes

In [55]:
pedigree_part1 = pedigree.loc[:,['Id','Sire','Dam']]
pedigree_part1

,Id,Sire,Dam
0,478665,0,0
1,478620,0,0
2,478601,02F49B,01FD38
3,478656,02F49B,01FD38
4,478671,02F49B,01FD38
...,...,...,...
8357,919218,597579,900730
8358,919013,597579,900730
8359,919448,597579,900730
8360,919246,597579,900730


In [56]:
pedigree_part2 = pedigree.loc[:,['Id','Year_Class','Selected_gen']]
pedigree_part2

,Id,Year_Class,Selected_gen
0,478665,2013,7
1,478620,2013,7
2,478601,2013,7
3,478656,2013,7
4,478671,2013,7
...,...,...,...
8357,919218,2017,8
8358,919013,2017,8
8359,919448,2017,8
8360,919246,2017,8


In [57]:
pedigree_part1.merge(
    pedigree_part2,on='Id').merge(
    pheno,left_on='Id',right_on='PIT',how='inner')

,Id,Sire,Dam,Year_Class,Selected_gen,PIT,Length,Weight,Tank,Sex,Site
0,916577,597579,479801,2017,8,916577,455.0,1556.0,3.0,U,1.0
1,915812,597579,479801,2017,8,915812,430.0,1339.0,3.0,M,1.0
2,915812,597579,479801,2017,8,915812,430.0,1339.0,3.0,M,1.0
3,916294,597579,479801,2017,8,916294,505.0,2038.0,2.0,M,1.0
4,916246,597579,479801,2017,8,916246,415.0,1213.0,1.0,U,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2842,918721,597579,900730,2017,8,918721,500.0,2168.0,4.0,U,2.0
2843,919049,597579,900730,2017,8,919049,530.0,2862.0,4.0,U,2.0
2844,918919,597579,900730,2017,8,918919,505.0,1888.0,4.0,U,2.0
2845,918637,597579,900730,2017,8,918637,430.0,1208.0,4.0,U,2.0


What if we want only the key combinations that are not found in the other dataframe. In `R` is known as `anti_join`

In [58]:
out_join = pedigree.merge(pheno, left_on='Id', right_on='PIT', how='outer', indicator=True)
out_join._merge

0        left_only
1        left_only
2        left_only
3        left_only
4        left_only
           ...    
8439    right_only
8440    right_only
8441    right_only
8442    right_only
8443    right_only
Name: _merge, Length: 8444, dtype: category
Categories (3, object): ['left_only', 'right_only', 'both']

In [59]:
merged_list = out_join.loc[out_join['_merge']=='left_only']
pedigree_only = pedigree[pedigree['Id'].isin(merged_list['Id'])]
pedigree_only

,Id,Sire,Dam,Year_Class,Selected_gen
0,478665,0,0,2013,7
1,478620,0,0,2013,7
2,478601,02F49B,01FD38,2013,7
3,478656,02F49B,01FD38,2013,7
4,478671,02F49B,01FD38,2013,7
...,...,...,...,...,...
8357,919218,597579,900730,2017,8
8358,919013,597579,900730,2017,8
8359,919448,597579,900730,2017,8
8360,919246,597579,900730,2017,8


In [60]:
merged_list = out_join.loc[out_join['_merge']=='right_only']
pheno_only = pheno[pheno['PIT'].isin(merged_list['PIT'])]
pheno_only

,PIT,Length,Weight,Tank,Sex,Site
6,9186524,535.0,3065.0,4.0,U,2.0
24,9168104,485.0,1840.0,3.0,U,1.0
29,9163844,430.0,1236.0,3.0,U,1.0
31,9169034,460.0,1390.0,3.0,M,1.0
36,9191934,445.0,1233.0,4.0,U,2.0
40,9196545,485.0,1713.0,1.0,U,1.0
388,9171292,475.0,1617.0,1.0,U,1.0
398,9202514,495.0,1955.0,1.0,M,1.0
759,916387,465.0,1526.0,2.0,U,1.0
1215,916384,430.0,1236.0,3.0,U,1.0


We can also use the index for merging dataframes

In [61]:
pheno_index = pheno.copy()
pheno_index = pheno_index.set_index('PIT')

In [62]:
pheno_index.head()

,Length,Weight,Tank,Sex,Site
PIT,,,,,
919540,465.0,1514.0,1.0,U,1.0
918025,455.0,1250.0,1.0,U,1.0
917803,405.0,937.0,1.0,U,1.0
918763,505.0,2667.0,4.0,M,2.0
917365,500.0,2204.0,4.0,U,2.0


In [63]:
combine_index = pd.merge(pheno_index, pedigree, left_index=True, right_on='Id', how='inner')
combine_index

,Length,Weight,Tank,Sex,Site,Id,Sire,Dam,Year_Class,Selected_gen
8273,465.0,1514.0,1.0,U,1.0,919540,597579,900730,2017,8
5874,455.0,1250.0,1.0,U,1.0,918025,480075,597462,2017,8
5676,405.0,937.0,1.0,U,1.0,917803,479785,479061,2017,8
8166,505.0,2667.0,4.0,M,2.0,918763,596713,480633,2017,8
6817,500.0,2204.0,4.0,U,2.0,917365,478795,479821,2017,8
...,...,...,...,...,...,...,...,...,...,...
7427,445.0,1377.0,2.0,U,1.0,917155,479994,900879,2017,8
6331,475.0,1565.0,2.0,U,1.0,920227,479024,597516,2017,8
4515,440.0,1300.0,2.0,U,1.0,915897,597426,480449,2017,8
6797,450.0,1510.0,2.0,U,1.0,917341,478795,479821,2017,8


In [64]:
pedigree_index = pedigree.copy()
pedigree_index = pedigree_index.set_index('Id')
out_index = pedigree_index.join(pheno_index, how='inner')
out_index

,Sire,Dam,Year_Class,Selected_gen,Length,Weight,Tank,Sex,Site
915581,597426,480449,2017,8,460.0,1470.0,2.0,U,1.0
915582,597179,480301,2017,8,450.0,1752.0,4.0,U,2.0
915583,900663,596937,2017,8,440.0,1436.0,2.0,U,1.0
915584,479924,597166,2017,8,465.0,1585.0,4.0,U,2.0
915587,479557,596943,2017,8,465.0,1594.0,4.0,U,2.0
...,...,...,...,...,...,...,...,...,...
920570,900741,479406,2017,8,505.0,2079.0,2.0,U,1.0
920571,480439,597565,2017,8,440.0,1487.0,2.0,M,1.0
920574,480535,480155,2017,8,470.0,1520.0,4.0,U,2.0
920577,480075,597462,2017,8,460.0,1313.0,3.0,M,1.0


## Exercises

### Exercise 1 

We will work with the `tilapia_pheno.txt` file.

* Sort the dataframe in terms of `Weight_final` and `Weight_initial` from higher to lower.
* Sort the dataframe based on the difference between `Weight_final` and `Weight_initial` from lower to higher.
* Calculate how many animals are in each Pond of each Location. Also calculate the mean `Weight_final` and `Length_final` and the corresponding standard deviations.
* Calculate how many animals of each line are in each happa of each pond and location. For all numeric columns estimate the mean, max, min and standard deviation.

### Exercise 2

We will work with the `tilapia_pedigree.txt` and the `tilapia_pheno.txt` files.

* Perform a left, right and an outer merge
* Create a new dataframe of the `tilapia_pedigree` file that does not contain the animals that are in the `tilapia_pheno` file.
* Create a new dataframe from the `tilapia_pheno` that contains only the animals that are in the `tilapia_pedigree`. Proceed by calculating the mean, max, min and standard deviation of the difference between `Weight_final` and `Weight_initial` for each line on both locations. 

### Exercise 3

We will work with the `variants.txt` file. The dataset contains information about the location of genetic variants across different chromosomes.

* Calculate the number of variants for each chromosome and plot the results in the form of a horizontal bar chart.
* For each chromosome estimate the base distribution of the Reference column. Find out which base is the most common in each chromosome. Do the same for the Alternative column.

### Exercise 4

We will work with the `brca_data.csv` file. The dataset contains gene expression data related to breast cancer.

* Change the format of the dataframe so that you have as an index the unique values of `PR.Status` column (the shape of your dataframe should be 5x46).
* Change the format of the dataframe so that all genes (columns starting with rs_) are in one columm named as `Gene`.
* Plot the 10 genes with the highest mean expression on positive patients in a bar chart. Do the same for negative patients.
* As above but this time plot the 10 genes with the lowest mean expression in a horizontal bar chart.